In [1]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('simvision_pixel_TDC.csv', skiprows=1) ## Skip first row SimTime = 0

In [ ]:
### Original column names
### 'SimTime',
### 'ETROC2_tb.ETROC2_inst.PixelMatrix_HTree_inst.PixelCol_inst_1.Pixel_inst_2.PixelAnalog_inst.TDC_CAL[9:0]'
### 'ETROC2_tb.ETROC2_inst.PixelMatrix_HTree_inst.PixelCol_inst_1.Pixel_inst_2.PixelAnalog_inst.TDC_TOA[9:0]'
### 'ETROC2_tb.ETROC2_inst.PixelMatrix_HTree_inst.PixelCol_inst_1.Pixel_inst_2.PixelAnalog_inst.TDC_TOT[8:0]'
### 'ETROC2_tb.ETROC2_inst.PixelMatrix_HTree_inst.PixelCol1_inst_14.Pixel1_inst_14.PixelAnalog1_inst.TDC_CAL[9:0]'
### 'ETROC2_tb.ETROC2_inst.PixelMatrix_HTree_inst.PixelCol1_inst_14.Pixel1_inst_14.PixelAnalog1_inst.TDC_TOA[9:0]'
### 'ETROC2_tb.ETROC2_inst.PixelMatrix_HTree_inst.PixelCol1_inst_14.Pixel1_inst_14.PixelAnalog1_inst.TDC_TOT[8:0]'

### Replace column names (only when TDC file has been opened)
df.columns = ['SimTime', 'Row1Col2_TDC_CAL', 'Row1Col2_TDC_TOA', 'Row1Col2_TDC_TOT', 
              'Row14Col14_TDC_CAL', 'Row14Col14_TDC_TOA', 'Row14Col14_TDC_TOT']

In [ ]:
df.head()

In [ ]:
### Covert hex to decimal
row1col2_cal = df['Row1Col2_TDC_CAL'].squeeze()
row1col2_toa = df['Row1Col2_TDC_TOA'].squeeze()
b16 = lambda x: int(x, 16)
row1col2_cal = row1col2_cal.apply(b16)
row1col2_toa = row1col2_toa.apply(b16)

In [ ]:
### plotting
import matplotlib.pyplot as plt
%matplotlib inline

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
fig.suptitle("1st row, 2nd column pixel TDC values", fontsize=14)
ax1.hist(row1col2_cal)
ax1.set_xlabel('CAL code', fontsize=12)

ax2.scatter(row1col2_toa, df['Row1Col2_TDC_TOT'])
ax2.grid()
ax2.set_xlabel('TOA code', fontsize=12)
ax2.set_ylabel('TOT code', fontsize=12)

In [ ]:
### Covert hex to decimal
row14col14_cal = df['Row14Col14_TDC_CAL'].squeeze()
row14col14_toa = df['Row14Col14_TDC_TOA'].squeeze()
row14col14_cal = row14col14_cal.apply(b16)
row14col14_toa = row14col14_toa.apply(b16)

In [ ]:
### plotting
import matplotlib.pyplot as plt
%matplotlib inline

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
fig.suptitle("14th row, 14th column pixel TDC values", fontsize=14)
ax1.hist(row14col14_cal)
ax1.set_xlabel('CAL code', fontsize=12)

ax2.scatter(row14col14_toa, df['Row14Col14_TDC_TOT'])
ax2.grid()
ax2.set_xlim(425, 438)
ax2.set_xlabel('TOA code', fontsize=12)
ax2.set_ylabel('TOT code', fontsize=12)

In [ ]:
df2 = pd.read_csv('simv ision_fastcmd_data.csv', skiprows=1)

In [ ]:
### Original column names
### 'SimTime' 
### 'ETROC2_tb.FCGenLocal_inst.FastComByte[7:0]'

### Replace column names (only when fast command file has been opened)
df2.columns = ['SimTime', 'FastComByte']
df2.head()

In [ ]:
df2['FastComByte']
### F0 (1111 0000): Idle
### 69 (0110 1001): Charge injection
### 5A (0101 1010): BCR (Bunch counter reset at BC0, reset the counter to zero)
### 96 (1001 0110): L1A
### Reference: https://indico.cern.ch/event/1127562/contributions/4904781/attachments/2454504/4319592/ETROCemulator%20slides20220921.pdf

In [ ]:
### create a list of our conditions
conditions = [
    (df2['FastComByte'] == 'F0'),
    (df2['FastComByte'] == '69'),
    (df2['FastComByte'] == '5A'),
    (df2['FastComByte'] == '96')
    ]

### create a list of the values we want to assign for each condition
values = ['Idle', 'Charge injection', 'BCR', 'L1A']

### create a new column and use np.select to assign values to it using our lists as arguments
df2['fcmd_defn'] = np.select(conditions, values)

### display updated DataFrame
print(df2)

In [2]:
df3 = pd.read_csv('simvision.csv', skiprows=1)
df3.columns = ['SimTime', 'DOR', 'DOL']
df3 = df3.drop(columns=['DOL'])
df3 = df3.iloc[:-1 , :] ## drop the last row

In [3]:
sim = df3.diff(periods=-1).fillna(0)
sim['SimTime'] = sim['SimTime'].abs()*0.001
sim['nth'] = sim['SimTime']/1560.
sim.head(10)

,SimTime,DOR,nth
0,1560.0,1.0,1.0
1,1560.0,-1.0,1.0
2,3120.0,1.0,2.0
3,23400.0,-1.0,15.0
4,1560.0,1.0,1.0
5,3120.0,-1.0,2.0
6,4680.0,1.0,3.0
7,1560.0,-1.0,1.0
8,1560.0,1.0,1.0
9,4680.0,-1.0,3.0


In [4]:
df3['diff'] = sim['SimTime'] ## Time difference between (nth, (n+1)th) row, n = 0, 1, 2, ...
df3['cycle'] = sim['nth']
df3.head(10)

,SimTime,DOR,diff,cycle
0,133809921000,1,1560.0,1.0
1,133811481000,0,1560.0,1.0
2,133813041000,1,3120.0,2.0
3,133816161000,0,23400.0,15.0
4,133839561000,1,1560.0,1.0
5,133841121000,0,3120.0,2.0
6,133844241000,1,4680.0,3.0
7,133848921000,0,1560.0,1.0
8,133850481000,1,1560.0,1.0
9,133852041000,0,4680.0,3.0


In [5]:
### Repeat rows in a pandas DataFrame based on column value
### https://stackoverflow.com/questions/47336704/repeat-rows-in-a-pandas-dataframe-based-on-column-value
bit_df = df3.reindex(df3.index.repeat(df3.cycle))
bit_df = bit_df.drop(columns=['SimTime', 'diff', 'cycle'])
bit_df = bit_df.iloc[20:, :] ## drop the first 20 rows
bit_df = bit_df.reset_index(drop=True)
bit_df.head(25)

,DOR
0,0
1,0
2,1
3,1
4,1
5,0
6,1
7,0
8,0
9,0


In [29]:
### 3C5C pattern recognition
### Only check whether the data has 3C5C pattern at anywhere or not
### 3 C 5 C (hex)
### 0011 1100 0101 1100 (binary)
pattern = [0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0] ## Since raw data bit is reversed!

matched = bit_df.rolling(len(pattern)).apply(lambda x: all(np.equal(x, pattern)), raw=True)
matched = matched.sum(axis = 1).astype(bool)   #Sum to perform boolean OR
idx_matched = np.where(matched)[0]
subset = [(match-len(pattern)+1, match+1, match+25) for match in idx_matched]
#subset = [range(match-len(pattern)+1, match+1) for match in idx_matched]   
#aligned_data = pd.concat([aligned_data.iloc[subs,:] for subs in subset], axis = 0)
#aligned_data.reset_index(inplace=True, drop=True)

In [31]:
output = []
tmp = 0

for subs in subset:
    bit16 = bit_df.iloc[range(subs[0], subs[1]), :].values.flatten().tolist()
    bit24 = bit_df.iloc[range(subs[1], subs[2]), :].values.flatten().tolist()
    if (subs[0]-tmp) != 40:
        print(tmp, subs[0])
        tmp = subs[0]
    bit16.reverse() ## Raw data bit order is reversed!
    bit24.reverse() ## Raw data bit order is reversed!
    s1 = ''.join([str(x) for x in bit16])
    s2 = ''.join([str(x) for x in bit24])
    output.append([s1, s2, format(int(s1, 2), 'X'), format(int(s2, 2), 'X')])

0 0
0 80
80 160
160 240
240 320
320 400
400 480
480 560
560 640
640 720
720 800
800 880
880 960
960 1040
1040 1120
1120 1200
1200 1280
1280 1360
1360 1440
1440 1520
1520 1600
1600 1680
1680 1760
1760 1840
1840 1920
1920 2000
2000 2080
2080 2160
2160 2240
2240 2320
2320 2400
2400 2480
2480 2560
2560 2640
2640 2720
2720 2800
2800 2880
2880 2960
2960 3040
3040 3120
3120 3200
3200 3280
3280 3360
3360 3440
3440 3520
3520 3600
3600 3680
3680 3760
3760 3840
3840 3920
3920 4000
4000 4080
4080 4160
4160 4240
4240 4320
4320 4400
4400 4480
4480 4560
4560 4640
4640 4720
4720 4800
4800 4880
4880 4960
4960 5040
5040 5120
5120 5200
5200 5280
5280 5360
5360 5440
5440 5520
5520 5600
5600 5680
5680 5760
5760 5840
5840 5920
5920 6000
6000 6080
6080 6160
6160 6240
6240 6320
6320 6400
6400 6480
6480 6560
6560 6640
6640 6720
6720 6800
6800 6880
6880 6960
6960 7040
7040 7120
7120 7200
7200 7280
7280 7360
7360 7440
7440 7520
7520 7600
7600 7680
7680 7760
7760 7840
7840 7920
7920 8000
8000 8080
8080 8160
8160 

122360 122440
122440 122520
122520 122600
122600 122680
122680 122760
122760 122840
122840 122920
122920 123000
123000 123080
123080 123160
123160 123240
123240 123320
123320 123400
123400 123480
123480 123560
123560 123640
123640 123720
123720 123800
123800 123880
123880 123960
123960 124040
124040 124120
124120 124200
124200 124280
124280 124360
124360 124440
124440 124520
124520 124600
124600 124680
124680 124760
124760 124840
124840 124920
124920 125000
125000 125080
125080 125160
125160 125240
125240 125320
125320 125400
125400 125480
125480 125560
125560 125640
125640 125720
125720 125800
125800 125880
125880 125960
125960 126040
126040 126120
126120 126200
126200 126280
126280 126360
126360 126440
126440 126520
126520 126600
126600 126680
126680 126760
126760 126840
126840 126920
126920 127000
127000 127080
127080 127160
127160 127240
127240 127320
127320 127400
127400 127480
127480 127560
127560 127640
127640 127720
127720 127800
127800 127880
127880 127960
127960 128040
128040

174680 174760
174760 174840
174840 174920
174920 175000
175000 175080
175080 175160
175160 175240
175240 175320
175320 175400
175400 175480
175480 175560
175560 175640
175640 175720
175720 175800
175800 175880
175880 175960
175960 176040
176040 176120
176120 176200
176200 176280
176280 176360
176360 176440
176440 176520
176520 176600
176600 176680
176680 176760
176760 176840
176840 176920
176920 177000
177000 177080
177080 177160
177160 177240
177240 177320
177320 177400
177400 177480
177480 177560
177560 177640
177640 177720
177720 177800
177800 177880
177880 177960
177960 178040
178040 178120
178120 178200
178200 178280
178280 178360
178360 178440
178440 178520
178520 178600
178600 178680
178680 178760
178760 178840
178840 178920
178920 179000
179000 179080
179080 179160
179160 179240
179240 179320
179320 179400
179400 179480
179480 179560
179560 179640
179640 179720
179720 179800
179800 179880
179880 179960
179960 180040
180040 180120
180120 180200
180200 180280
180280 180360
180360

334040 334120
334120 334200
334200 334280
334280 334360
334360 334440
334440 334520
334520 334600
334600 334680
334680 334760
334760 334840
334840 334920
334920 335000
335000 335080
335080 335160
335160 335240
335240 335320
335320 335400
335400 335480
335480 335560
335560 335640
335640 335720
335720 335800
335800 335880
335880 335960
335960 336040
336040 336120
336120 336200
336200 336280
336280 336360
336360 336440
336440 336520
336520 336600
336600 336680
336680 336760
336760 336840
336840 336920
336920 337000
337000 337080
337080 337160
337160 337240
337240 337320
337320 337400
337400 337480
337480 337560
337560 337640
337640 337720
337720 337800
337800 337880
337880 337960
337960 338040
338040 338120
338120 338200
338200 338280
338280 338360
338360 338440
338440 338520
338520 338600
338600 338680
338680 338760
338760 338840
338840 338920
338920 339000
339000 339080
339080 339160
339160 339240
339240 339320
339320 339400
339400 339480
339480 339560
339560 339640
339640 339720
339720

967400 967480
967480 967560
967560 967640
967640 967720
967720 967800
967800 967880
967880 967960
967960 968040
968040 968120
968120 968200
968200 968280
968280 968360
968360 968440
968440 968520
968520 968600
968600 968680
968680 968760
968760 968840
968840 968920
968920 969000
969000 969080
969080 969160
969160 969240
969240 969320
969320 969400
969400 969480
969480 969560
969560 969640
969640 969720
969720 969800
969800 969880
969880 969960
969960 970040
970040 970120
970120 970200
970200 970280
970280 970360
970360 970440
970440 970520
970520 970600
970600 970680
970680 970760
970760 970840
970840 970920
970920 971000
971000 971080
971080 971160
971160 971240
971240 971320
971320 971400
971400 971480
971480 971560
971560 971640
971640 971720
971720 971800
971800 971880
971880 971960
971960 972040
972040 972120
972120 972200
972200 972280
972280 972360
972360 972440
972440 972520
972520 972600
972600 972680
972680 972760
972760 972840
972840 972920
972920 973000
973000 973080
973080

...

1331235 1331315
1331315 1331395
1331395 1331475
1331475 1331555
1331555 1331635
1331635 1331715
1331715 1331795
1331795 1331875
1331875 1331955
1331955 1332035
1332035 1332115
1332115 1332195
1332195 1332275
1332275 1332355
1332355 1332435
1332435 1332515
1332515 1332595
1332595 1332675
1332675 1332755
1332755 1332835
1332835 1332915
1332915 1332995
1332995 1333075
1333075 1333155
1333155 1333235
1333235 1333315
1333315 1333395
1333395 1333475
1333475 1333555
1333555 1333635
1333635 1333715
1333715 1333795
1333795 1333875
1333875 1333955
1333955 1334035
1334035 1334115
1334115 1334195
1334195 1334275
1334275 1334355
1334355 1334435
1334435 1334515
1334515 1334595
1334595 1334675
1334675 1334755
1334755 1334835
1334835 1334915
1334915 1334995
1334995 1335075
1335075 1335155
1335155 1335235
1335235 1335315
1335315 1335395
1335395 1335475
1335475 1335555
1335555 1335635
1335635 1335715
1335715 1335795
1335795 1335875
1335875 1335955
1335955 1336035
1336035 1336115
1336115 1336195
1336195 

1588915 1588995
1588995 1589075
1589075 1589155
1589155 1589235
1589235 1589315
1589315 1589395
1589395 1589475
1589475 1589555
1589555 1589635
1589635 1589715
1589715 1589795
1589795 1589875
1589875 1589955
1589955 1590035
1590035 1590115
1590115 1590195
1590195 1590275
1590275 1590355
1590355 1590435
1590435 1590515
1590515 1590595
1590595 1590675
1590675 1590755
1590755 1590835
1590835 1590915
1590915 1590995
1590995 1591075
1591075 1591155
1591155 1591235
1591235 1591315
1591315 1591395
1591395 1591475
1591475 1591555
1591555 1591635
1591635 1591715
1591715 1591795
1591795 1591875
1591875 1591955
1591955 1592035
1592035 1592115
1592115 1592195
1592195 1592275
1592275 1592355
1592355 1592435
1592435 1592515
1592515 1592595
1592595 1592675
1592675 1592755
1592755 1592835
1592835 1592915
1592915 1592995
1592995 1593075
1593075 1593155
1593155 1593235
1593235 1593315
1593315 1593395
1593395 1593475
1593475 1593555
1593555 1593635
1593635 1593715
1593715 1593795
1593795 1593875
1593875 

1636035 1636115
1636115 1636195
1636195 1636275
1636275 1636355
1636355 1636435
1636435 1636515
1636515 1636595
1636595 1636675
1636675 1636755
1636755 1636835
1636835 1636915
1636915 1636995
1636995 1637075
1637075 1637155
1637155 1637235
1637235 1637315
1637315 1637395
1637395 1637475
1637475 1637555
1637555 1637635
1637635 1637715
1637715 1637795
1637795 1637875
1637875 1637955
1637955 1638035
1638035 1638115
1638115 1638195
1638195 1638275
1638275 1638355
1638355 1638435
1638435 1638515
1638515 1638595
1638595 1638675
1638675 1638755
1638755 1638835
1638835 1638915
1638915 1638995
1638995 1639075
1639075 1639155
1639155 1639235
1639235 1639315
1639315 1639395
1639395 1639475
1639475 1639555
1639555 1639635
1639635 1639715
1639715 1639795
1639795 1639875
1639875 1639955
1639955 1640035
1640035 1640115
1640115 1640195
1640195 1640275
1640275 1640355
1640355 1640435
1640435 1640515
1640515 1640595
1640595 1640675
1640675 1640755
1640755 1640835
1640835 1640915
1640915 1640995
1640995 

1722035 1722115
1722115 1722195
1722195 1722275
1722275 1722355
1722355 1722435
1722435 1722515
1722515 1722595
1722595 1722675
1722675 1722755
1722755 1722835
1722835 1722915
1722915 1722995
1722995 1723075
1723075 1723155
1723155 1723235
1723235 1723315
1723315 1723395
1723395 1723475
1723475 1723555
1723555 1723635
1723635 1723715
1723715 1723795
1723795 1723875
1723875 1723955
1723955 1724035
1724035 1724115
1724115 1724195
1724195 1724275
1724275 1724355
1724355 1724435
1724435 1724515
1724515 1724595
1724595 1724675
1724675 1724755
1724755 1724835
1724835 1724915
1724915 1724995
1724995 1725075
1725075 1725155
1725155 1725235
1725235 1725315
1725315 1725395
1725395 1725475
1725475 1725555
1725555 1725635
1725635 1725715
1725715 1725795
1725795 1725875
1725875 1725955
1725955 1726035
1726035 1726115
1726115 1726195
1726195 1726275
1726275 1726355
1726355 1726435
1726435 1726515
1726515 1726595
1726595 1726675
1726675 1726755
1726755 1726835
1726835 1726915
1726915 1726995
1726995 

2020355 2020435
2020435 2020515
2020515 2020595
2020595 2020675
2020675 2020755
2020755 2020835
2020835 2020915
2020915 2020995
2020995 2021075
2021075 2021155
2021155 2021235
2021235 2021315
2021315 2021395
2021395 2021475
2021475 2021555
2021555 2021635
2021635 2021715
2021715 2021795
2021795 2021875
2021875 2021955
2021955 2022035
2022035 2022115
2022115 2022195
2022195 2022275
2022275 2022355
2022355 2022435
2022435 2022515
2022515 2022595
2022595 2022675
2022675 2022755
2022755 2022835
2022835 2022915
2022915 2022995
2022995 2023075
2023075 2023155
2023155 2023235
2023235 2023315
2023315 2023395
2023395 2023475
2023475 2023555
2023555 2023635
2023635 2023715
2023715 2023795
2023795 2023875
2023875 2023955
2023955 2024035
2024035 2024115
2024115 2024195
2024195 2024275
2024275 2024355
2024355 2024435
2024435 2024515
2024515 2024595
2024595 2024675
2024675 2024755
2024755 2024835
2024835 2024915
2024915 2024995
2024995 2025075
2025075 2025155
2025155 2025235
2025235 2025315
2025315 

In [25]:
new_df = pd.DataFrame(columns=['bit16', 'bit24', 'bit16_hex', 'bit24_hex'], data=output)
new_df.to_csv('result1.csv', index=False)

In [ ]:
subset1 = []
subset2 = []

output = []
for i in range(int(bit_df.shape[0]/40)):
    x1, x2 = 0+40*i, 16+40*i
    y1, y2 = 16+40*i, 40+40*i
    bit16 = bit_df.iloc[range(x1, x2), :].values.flatten().tolist()
    bit24 = bit_df.iloc[range(y1, y2), :].values.flatten().tolist()
    bit16.reverse() ## Raw data bit order is reversed!
    bit24.reverse() ## Raw data bit order is reversed!
    s1 = ''.join([str(x) for x in bit16])
    s2 = ''.join([str(x) for x in bit24])
    output.append([s1, s2, format(int(s1, 2), 'X'), format(int(s2, 2), 'X')])
    
    if i < 10:
        print(s1, s2, format(int(s1, 2), 'X'), format(int(s2, 2), 'X'))

In [ ]:
new_df = pd.DataFrame(columns=['bit16', 'bit24', 'bit16_hex', 'bit24_hex'], data=output)
new_df.to_csv('result1.csv', index=False)

In [26]:
### 6AF3 pattern
### 6 A F 3
### 0110 1010 1111 0011
pattern = [1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0] ## Since raw data bit is reversed!

matched = bit_df.rolling(len(pattern)).apply(lambda x: all(np.equal(x, pattern)), raw=True)
matched = matched.sum(axis = 1).astype(bool)   #Sum to perform boolean OR
idx_matched = np.where(matched)[0]
subset = [(match-len(pattern)+1, match+1, match+25) for match in idx_matched]